In [1]:
import time
import datetime
principio = datetime.datetime.now()
principio

datetime.datetime(2024, 1, 9, 20, 24, 56, 33689)

In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.ets import AutoETS
from sktime.utils.plotting import plot_series
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.performance_metrics.forecasting import MeanSquaredError
from sklearn.model_selection import GridSearchCV
from sktime.transformations.series.boxcox import LogTransformer


# Read Data
data = pd.read_csv('../data/datos sin días.csv')
data.set_index('Fecha', inplace=True)
data['Billetes'] = data['Billetes'].astype('float64')
data.index = pd.to_datetime(data['Billetes'].index)
data['Billetes'].astype('float64').to_period('D')


Bad key savefig.frameon in file C:\Users\manue\anaconda3\envs\TecnicasPrediccion\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 421 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.4.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.level in file C:\Users\manue\anaconda3\envs\TecnicasPrediccion\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 472 ('verbose.level  : silent      # one of silent, helpful, debug, debug-annoying')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.4.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.fileo in file C:\Users\manue\anaconda3\envs\TecnicasPrediccion\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 473 ('verbose.fileo  : sys.stdout  # a log filename, sys.std

Fecha
2013-01-01     2338.0
2013-01-02     9376.0
2013-01-03     9190.0
2013-01-04     9523.0
2013-01-05     5492.0
               ...   
2016-12-27    23928.0
2016-12-28    23960.0
2016-12-29    24279.0
2016-12-30    22763.0
2016-12-31     5346.0
Freq: D, Name: Billetes, Length: 1461, dtype: float64

In [3]:
# transformer = LogTransformer()
# log_data = transformer.fit_transform(data)
# log_data.tail()

In [4]:
y_train, y_test = temporal_train_test_split(y = data['Billetes'].astype('float64').to_period('d'), test_size=31)

In [5]:
from itertools import product

fh = np.arange(len(y_test)) + 1

# Definir las opciones para cada parámetro
error_options = ["additive", "multiplicative"]
trend_options = ["additive", "multiplicative"]
seasonal_options = ["additive", "multiplicative"]

# Generar todas las combinaciones posibles
param_combinations = list(product(error_options, trend_options, seasonal_options))

# Probar diferentes combinaciones
for params in param_combinations:
    autoETS_model = AutoETS(
        error=params[0],
        trend=params[1],
        seasonal=params[2],
        sp=7,
        n_jobs=-1
    )
    autoETS_model.fit(y_train)
    ETS_pred = autoETS_model.predict(fh)
    mape = mean_absolute_percentage_error(y_test, ETS_pred)
    print(f"Parameters: {params}, MAPE: {mape}")

Parameters: ('additive', 'additive', 'additive'), MAPE: 0.4398070727518802
Parameters: ('additive', 'additive', 'multiplicative'), MAPE: 0.40567354907664943
Parameters: ('additive', 'multiplicative', 'additive'), MAPE: 0.2646999682186132
Parameters: ('additive', 'multiplicative', 'multiplicative'), MAPE: 0.37231152448117677
Parameters: ('multiplicative', 'additive', 'additive'), MAPE: 0.3796339606793976
Parameters: ('multiplicative', 'additive', 'multiplicative'), MAPE: 0.3006750719106111
Parameters: ('multiplicative', 'multiplicative', 'additive'), MAPE: 0.2785370661003324
Parameters: ('multiplicative', 'multiplicative', 'multiplicative'), MAPE: 0.30171913471604667


In [6]:
fh = np.arange(1, 32)
data_auto_model = AutoETS(
        error="additive",
        trend="multiplicative",
        seasonal="additive",
        sp=7,
        n_jobs=-1
    )

In [7]:
data_auto_model.fit(y_train)

AutoETS(error='additive', n_jobs=-1, seasonal='additive', sp=7,
        trend='multiplicative')

In [8]:
y_pred = data_auto_model.predict(fh)

In [9]:
mape = mean_absolute_percentage_error(y_test[0:31], y_pred[0:31])
mse = MeanSquaredError()
mse = mse(y_test[0:31], y_pred[0:31])
r2 = r2_score(list(y_test[0:31]), list(y_pred[0:31]))

print(f'MAPE: {mape}')
print(f'MSE: {mse}')
print(f'R2 Score: {r2}')


MAPE: 0.2646999682186132
MSE: 27020314.698644344
R2 Score: 0.5927001089375472


In [10]:
final = datetime.datetime.now()
duracion = final-principio
print(principio)
print(final)
print(duracion)

2024-01-09 20:24:56.033689
2024-01-09 20:24:59.843762
0:00:03.810073
